In [134]:
def matrix_print(Title, M):
    print(Title)
    for row in M:
        print([round(x,3)+0 for x in row])

def matrix_print_two(Action, Title1, M1, Title2, M2):
    print(Action)
    print(Title1, '\t'*int(len(M1)/2)+"\t"*len(M1), Title2)
    for i in range(len(M1)):
        row1 = ['{0:+7.3f}'.format(x) for x in M1[i]]
        row2 = ['{0:+7.3f}'.format(x) for x in M2[i]]
        print(row1,'\t', row2)

def matrix_zeros(rows, cols):
    A = []
    for i in range(rows):
        A.append([])
        for j in range(cols):
            A[-1].append(0.0)

    return A

def matrix_check_squareness(A):
    if len(A) != len(A[0]):
        raise ArithmeticError("Matrix must be square to inverse.")

def matrix_determinant(A, total=0):
    indices = list(range(len(A)))
    
    if len(A) == 2 and len(A[0]) == 2:
        val = A[0][0] * A[1][1] - A[1][0] * A[0][1]
        return val

    for fc in indices:
        As = matrix_copy(A)
        As = As[1:]
        height = len(As)
        builder = 0

        for i in range(height):
            As[i] = As[i][0:fc] + As[i][fc+1:]

        sign = (-1) ** (fc % 2)
        sub_det = matrix_determinant(As)
        total += A[0][fc] * sign * sub_det

    return total

def matrix_check_non_singular(A):
    det = matrix_determinant(A)
    if det != 0:
        return det
    else:
        raise ArithmeticError("Singular Matrix!")

def matrix_check_equality(A,B, tol=None):
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        return False

    for i in range(len(A)):
        for j in range(len(A[0])):
            if tol == None:
                if A[i][j] != B[i][j]:
                    return False
            else:
                if round(A[i][j],tol) != round(B[i][j],tol):
                    return False

    return True

        
def matrix_multiplication(matrix_a, matrix_b):
    rows_a = len(matrix_a)
    cols_a = len(matrix_a[0])
    rows_b = len(matrix_b)
    cols_b = len(matrix_b[0])

    if cols_a != rows_b:
      print("Incorrect dimensions.")
      return

    result = [[0 for row in range(cols_b)] for col in range(rows_a)] #creating result

    for i in range(rows_a):
        for j in range(cols_b):
            for k in range(cols_a):
                result[i][j] += matrix_a[i][k] * matrix_b[k][j]
    return result

def matrix_inverse(matrix_a, tol=None):
    # Section 1: Make sure matrix_a can be inverted.
    matrix_check_squareness(matrix_a)
    matrix_check_non_singular(matrix_a)

    # Section 2: Make copies of matrix_a & I, AM & IM, to use for row ops
    n = len(matrix_a)
    AM = matrix_copy(matrix_a)
    I = matrix_identity(n)
    IM = matrix_copy(I)

    # Section 3: Perform row operations
    indices = list(range(n)) # to allow flexible row referencing ***
    for fd in range(n): # fd stands for focus diagonal
        fdScaler = 1.0 / AM[fd][fd]
        # FIRST: scale fd row with fd inverse. 
        for j in range(n): # Use j to indicate column looping.
            AM[fd][j] *= fdScaler
            IM[fd][j] *= fdScaler
        # SECOND: operate on all rows except fd row as follows:
        for i in indices[0:fd] + indices[fd+1:]: 
            # *** skip row with fd in it.
            crScaler = AM[i][fd] # cr stands for "current row".
            for j in range(n): 
                # cr - crScaler * fdRow, but one element at a time.
                AM[i][j] = AM[i][j] - crScaler * AM[fd][j]
                IM[i][j] = IM[i][j] - crScaler * IM[fd][j]

    # Section 4: Make sure IM is an inverse of matrix_a with specified tolerance
    if matrix_check_equality(I, matrix_multiplication(matrix_a,IM), tol):
        return IM
    else:
        raise ArithmeticError("Matrix inverse out of tolerance.")

def matrix_identity(n):
    matrix_iden = matrix_zeros(n, n)
    for i in range(n):
        matrix_iden[i][i] = 1
    return matrix_iden

def matrix_transpose(matrix):
    rows = len(matrix)
    cols = len(matrix[0])

    matrixtrans = matrix_zeros(cols, rows)

    for i in range(rows):
        for j in range(cols):
            matrixtrans[j][i] = matrix[i][j]

    return matrixtrans

def matrix_copy(matrix):
    rows = len(matrix)
    cols = len(matrix[0])

    matrixcp = matrix_zeros(rows, cols)

    for i in range(rows):
        for j in range(cols):
            matrixcp[i][j] = matrix[i][j]

    return matrixcp

def matrix_add_begin_column(matrix, value):
    rows = len(matrix)
    cols = len(matrix[0])
    newcols = len(matrix[0]) + 1
    newmatrix = matrix_zeros(rows, newcols)
    
    for i in range(rows):
        newmatrix[i][0] = value
    
    for i in range(rows):
        for j in range(cols):
            newmatrix[i][j + 1] = matrix[i][j]
    
    return newmatrix

def matrix_add_end_column_potential_last(matrix, potential):
    rows = len(matrix)
    cols = len(matrix[0])
    newcols = len(matrix[0]) + 1
    newmatrix = matrix_zeros(rows, newcols)
    
    for i in range(rows):
        newmatrix[i][-1] = matrix[i][-1]**potential
    
    for i in range(rows):
        for j in range(cols):
            newmatrix[i][j] = matrix[i][j]
    
    return newmatrix
    
def logistic_regression_one_degree(matrix, y): # B = (X^T * X)^-1 * X^t * y
    newmatrix = matrix_add_begin_column(matrix, 1)
    matrixtranspose = matrix_transpose(newmatrix)
    section_1 = matrix_inverse(matrix_multiplication(matrixtranspose, newmatrix), 1) # (X^T * X)^-1
    section_2 = matrix_multiplication(section_1, matrixtranspose) # (X^T * X)^-1 * X^t
    section_3 = matrix_multiplication(section_2, y) # (X^T * X)^-1 * X^t * y
    
    return section_3

def logistic_regression_two_degree(matrix, y): # B = (X^T * X)^-1 * X^t * y
    newmatrix = matrix_add_end_column_potential_last(matrix_add_begin_column(matrix, 1), 2)
    matrixtranspose = matrix_transpose(newmatrix)
    section_1 = matrix_inverse(matrix_multiplication(matrixtranspose, newmatrix), 1) # (X^T * X)^-1
    section_2 = matrix_multiplication(section_1, matrixtranspose) # (X^T * X)^-1 * X^t
    section_3 = matrix_multiplication(section_2, y) # (X^T * X)^-1 * X^t * y
    
    return section_3

def logistic_regression_test():
    matrix = [[69], [67], [71], [65], [72], [68], [74], [65], [66], [72]]
    y = [[9.5], [8.5], [11.5], [10.5], [11], [7.5], [12], [7], [7.5], [13]]
    matrix_print("logistic_regression_one_degree", logistic_regression_one_degree(matrix, y))
    
    matrix2 = [[1900], [1910], [1920], [1930], [1940], [1950], [1960], [1970], [1980], [1990], [2000]]
    y2 = [[75.9950], [91.9720], [105.7110], [123.2030], [131.6690], [150.6970], [179.3230], [203.2120], [226.5050], [249.6330], [281.4220]]
    matrix_print("logistic_regression_two_degree", logistic_regression_two_degree(matrix2, y2))

In [136]:
logistic_regression_test()

logistic_regression_one_degree
[-25.651]
[0.515]
logistic_regression_two_degree
[32294.017]
[-34.987]
[0.009]
